# ASSIGNMENT 5

In [1]:
import pandas as pd
import numpy as np

from scipy.spatial.distance import cosine
pd.set_option('display.float_format', None) # reset the display format
np.seterr(all='warn')
import warnings
warnings.filterwarnings('error')

In [2]:
radio = pd.read_csv('radio_songs.csv')
radio.head()

,user,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,33,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,51,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
radio.shape

(100, 285)

# 1. Collaborative Filtering

### A. Recommend 10 songs to users who have listened to 'u2' and 'pink floyd'. Use item-item collaborative filtering to find songs that are similar using spatial distance with cosine. Since this measures the distance you need to subtract from 1 to get similarity as shown below.
```
from scipy.spatial.distance import cosine

similarity = 1 - cosine(item1, item2)
```

In [4]:
similarity_matrix = pd.DataFrame(columns=radio.columns, index=radio.columns)
similarity_matrix = similarity_matrix.drop(['user'], axis=1).drop(['user'], axis=0)
for i in similarity_matrix.columns:
    for j in similarity_matrix.index:
        try:
            similarity_matrix.at[j, i] = 1 - cosine(radio[i], radio[j])
        except:
            similarity_matrix.at[j, i] = 0

average_matrix = similarity_matrix[['u2', 'pink floyd']].copy()
average_matrix = average_matrix.drop(['u2', 'pink floyd'], axis=0)
average_matrix = average_matrix[(average_matrix['u2'] != 0) & (average_matrix['pink floyd'] != 0)]
average_matrix['Average cosine'] = (average_matrix['u2'] + average_matrix['pink floyd'])/2
print("10 Songs to recommend to users who have listened to U2 AND Pink Floyd:")
pd.DataFrame(average_matrix['Average cosine'].sort_values(ascending=False).head(10)).reset_index(drop=False).rename(columns={'index':'Song', 'Average cosine': 'Similarity Score'}).style.hide(axis='index')

10 Songs to recommend to users who have listened to U2 AND Pink Floyd:


Song,Similarity Score
robbie williams,0.394338
johnny cash,0.278839
audioslave,0.240903
foo fighters,0.237795
pearl jam,0.219913
incubus,0.155502


### B. Find user most similar to user 1606. Use user-user collaborative filtering with cosine similarity. List the recommended songs for user 1606 (Hint: find the songs listened to by the most similar user).

In [5]:
radio_user = radio.T
radio_user.columns = radio['user']
radio_user = radio_user.drop(['user'], axis=0)
user_matrix = pd.DataFrame(columns=radio['user'].values, index=radio['user'].values)
for i in user_matrix.columns:
    for j in user_matrix.index:
        try:
            user_matrix.at[j, i] = 1 - cosine(radio_user[i], radio_user[j])
        except:
            user_matrix.at[j, i] = 0
similar_user = user_matrix[1606].sort_values(ascending=False).index[1]
print("User " + str(similar_user) + " is most similar to User 1606 with similarity score of " + str(user_matrix[1606].sort_values(ascending=False)[similar_user]))
print("The songs recommended to User 1606 based on User " + str(similar_user) + ":")
pd.DataFrame(radio_user[radio_user[similar_user] == 1][similar_user]).reset_index(drop=False).rename(columns={'index':'Song'}).style.hide(axis='index')

User 1144 is most similar to User 1606 with similarity score of 0.27735009811261446
The songs recommended to User 1606 based on User 1144:


Song,1144
beastie boys,1
bob dylan,1
bob marley & the wailers,1
david bowie,1
elvis presley,1
eric clapton,1
johnny cash,1
pearl jam,1
pink floyd,1
the beatles,1


### C. How many of the recommended songs has already been listened to by user 1606?

In [6]:
user_1606 = radio_user[radio_user[1606] == 1][1606].index.to_list()
user_sim = radio_user[radio_user[similar_user] == 1][similar_user].index.to_list()
common = list(set(user_1606).intersection(set(user_sim)))
print("The number of recomended songs already listened to by user 1606 is " + str(len(common)) + ", which are: ")
for i in common:
    print(i)

The number of recomended songs already listened to by user 1606 is 2, which are: 
elvis presley
the beatles


### D. Use a combination of user-item approach to build a recommendation score for each song for each user using the following steps for each user-
- For each song for the user row, get the top 10 similar songs and their similarity score.
- For each of the top 10 similar songs, get a list of the user purchases
- Calculate a recommendation score as follows: 
- What are the top 5 song recommendations for user 1606?

In [7]:
r = radio.set_index('user')
# Get the top 10 similar songs and their similarity score for each song
top_similar_songs = {}
for song in radio.columns:
    if song != 'user':
        similar_songs = similarity_matrix[song].sort_values(ascending=False)[1:11]
        top_similar_songs[song] = dict(similar_songs)

# Calculate the recommendation score for each song for each user
recommendation_scores = pd.DataFrame(index=radio['user'], columns=similarity_matrix.columns)
for user in radio['user']:
    for song in similarity_matrix.columns:
        if song != 'user':
            similar_songs = top_similar_songs[song]
            similarity_scores = np.array(list(similar_songs.values()))
            purchase_history = r.loc[user, list(similar_songs.keys())]
            try:
                recommendation_score = (purchase_history * similarity_scores).sum() / similarity_scores.sum()
            except:
                recommendation_score = 0
            recommendation_scores.loc[user, song] = recommendation_score

# Get the top 5 song recommendations for user 1606
top_recommendations = recommendation_scores.loc[1606].sort_values(ascending=False)[:5]
print("Top 5 recommendations for User 1606 are:")
pd.DataFrame(top_recommendations).reset_index(drop=False).rename(columns={'index':'Song', 1606: 'Recommendation Score with User 1606'}).style.hide(axis='index')

Top 5 recommendations for User 1606 are:


Song,Recommendation Score with User 1606
elvis presley,0.289328
abba,0.239023
eric clapton,0.202740
frank sinatra,0.201139
howard shore,0.171749


# 2. Conceptual questions:

### 1. Name 2 other similarity measures that you can use instead of cosine similarity above.

The 2 other similarity measures that can be used instead of cosine similarity are:
- Pearson Correlation: Pearson correlation measures the linear correlation between two variables. It ranges from -1 to 1, with 1 representing a perfect positive correlation, 0 representing no correlation, and -1 representing a perfect negative correlation.
- Euclidean Distance: Euclidean distance measures the straight-line distance between two points in a multi-dimensional space.

### 2. What is needed to build a Content-Based Recommender system?

To build a Content-Based Recommender system, the following are needed:
- Item profiles: A representation of each item in the system based on features or attributes. For example, if recommending books, the item profile could include features such as genre, author, title, release year, etc.
- User profiles: A representation of each user in the system based on their purchases or reviews.
- Similarity measure: A method to calculate the similarity between item and user profiles. This could be based on cosine similarity, Pearson correlation, Euclidean distance, or other distance metrics.
- Recommendation algorithm: A method to generate recommendations based on the similarity between user and item profiles. This could include ranking items by similarity score, using machine learning models such as decision trees or neural networks, or other approaches.
- Evaluation metrics: A way to measure the effectiveness of the recommender system. This could include metrics such as accuracy, precision, recall, or others, depending on the specific goals and objectives of the system.

### 3. Name 2 methods to evaluate your recommender system.

- Precision: Precision is defined as the fraction of relevant recommendations among the total number of recommendations made by the system. In other words, it measures how many of the items recommended by the system are actually relevant to the user. A high precision indicates that the system is good at making relevant recommendations, while a low precision means that the system is making a lot of irrelevant recommendations.
- Recall: Recall is defined as the fraction of relevant items that are recommended by the system, among the total number of relevant items. In other words, it measures how many of the relevant items were actually recommended by the system. A high recall indicates that the system is good at capturing all the relevant items, while a low recall means that the system is missing many relevant items.